# Datu tīrīšana


Apstrādāšu Siguldas novada pašvaldības ceļu un ielu sarakstu, kas ir sadalīts četrās Excel darblapās: A ceļi, B ceļi, C ceļi un Ielas. Katra no šīm darblapām satur vairākas tabulas ar virsrakstiem un kopsavilkumiem, kur ceļi un ielas ir sadalīti pēc teritoriālā iedalījuma. Daudzi ceļi un ielas ir vēl sīkāk sadalīti posmos, kuros nosaukums ir norādīts tikai pirmajam posmam.

Apvienošu visus datus vienā tabulā, izdalīšu teritoriālo iedalījumu atsevišķā kolonnā, identificēšu atzarus un katram ceļam vai ielai piešķiršu unikālu ID (jo nosaukumi mēdz atkārtoties).

**Dati XLSX formātā pieejami šeit:** https://www.sigulda.lv/public/download.php?id=6266 

In [20]:
import pandas as pd

In [21]:
# ielādēju datus no katras darblapas, atlasot tikai nepieciešamās kolonnas un piešķirot tām nosaukumus.
# neiekļauju rindas, kur visas vērtības ir tukšas.

url = 'https://www.sigulda.lv/public/download.php?id=6266'
kol_nosaukumi = ['nosaukums', 'uzturesanas_klase', 'virsraksts', 'garums', 'segums']

df1 = pd.read_excel(url, 
                    sheet_name='A ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df2 = pd.read_excel(url, 
                    sheet_name='B ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df3 = pd.read_excel(url, 
                    sheet_name='C ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df4 = pd.read_excel(url, 
                    sheet_name='Ielas', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')


# izveidoju jaunu kolonnu ceļu un ielu grupai
df1.insert(1, 'grupa', 'A ceļš')
df2.insert(1, 'grupa', 'B ceļš')
df3.insert(1, 'grupa', 'C ceļš')
df4.insert(1, 'grupa', 'Iela')


# apvienoju datus vienā DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [22]:
# no kolonnas "virsraksts" izvelku teritorijas nosaukumu (divus pēdējos vārdus) un 
# saglabāju jaunā kolonnā "teritorija"

split = df['virsraksts'].str.rsplit(n=2)
df['teritorija'] = split.str[1] + " " + split.str[2]
df = df.drop('virsraksts', axis=1)


# apskatu unikālās vērtības kolonnā "teritorija"
df.teritorija.unique()

array(['Siguldas pagastā', 'raksturojošie parametri', nan,
       'Allažu pagastā', 'Mālpils pagastā', 'Inčukalna pagastā',
       'Krimuldas pagastā', 'Lēdurgas pagastā', 'Rihards Keišs',
       'uzvārds )', 'domes izpilddirektors', 'Mores pagastā',
       'Siguldas pilsētā', 'pilsētā (Peltēs)', 'pilsētā (Ķiparos)',
       'pilsētā (Kalnabeitēs)', 'Matiņu ciemā', 'Vējupītes ciemā',
       'Jūdažu ciemā', 'Mores ciemā', 'Allažu ciemā',
       '(Siguldas pagastā)', 'Mālpils ciemā', 'Upmalu ciemā',
       'Sidgundas ciemā', 'Inčukalna ciemā', 'Gaujas ciemā',
       '(Inčukalna pagastā)', 'Krustiņu ciemā', 'Kļavu ciemā',
       'Raganas ciemā', 'Inciema ciemā', 'Sunīšu ciemā', 'Turaidas ciemā',
       'Lēdurgas ciemā', 'Lodes ciemā'], dtype=object)

In [23]:
# koriģēju locījumus: "pagastā" → "pagasts", "ciemā" → "ciems"
df['teritorija'] = df['teritorija'].replace(r'(.*)pagastā$', r'\1pagasts', regex=True)
df['teritorija'] = df['teritorija'].replace(r'(.*)ciemā$', r'\1ciems', regex=True)


# koriģēju pilsētu un to rajonu nosaukumus
df['teritorija'] = df['teritorija'].replace({'Siguldas pilsētā': 'Siguldas pilsēta', 
                                             'pilsētā (Peltēs)': 'Siguldas pilsēta (Peltes)', 
                                             'pilsētā (Ķiparos)': 'Siguldas pilsēta (Ķipari)', 
                                             'pilsētā (Kalnabeitēs)': 'Siguldas pilsēta (Kalnabeites)'})


# koriģēju Egļupes ciema nosaukumus (atrodas divos pagastos)
df['teritorija'] = df['teritorija'].replace({'(Siguldas pagastā)': 'Egļupes ciems (Allažu pag.)', 
                                             '(Inčukalna pagastā)': 'Egļupes ciems (Inčukalna pag.)'})


In [24]:
# izdzēšu visus neatbilstošos ierakstus kolonnā "teritorija":
# kur nav norādīts "pagasts", "pilsēta" vai "ciems"
df['teritorija'] = df['teritorija'].replace(r'^(?!.*pagasts|.*pilsēta|.*ciems).*$', pd.NA, regex=True)


# aizpildu tukšās vērtības ar iepriekšējās rindas vērtību
df['teritorija'] = df['teritorija'].ffill()

In [25]:
# apskatu unikālās uzturēšanas klases
df.uzturesanas_klase.unique()

array([nan, 'Uzturēšanas klase', 'C', 'D', 'B'], dtype=object)

In [26]:
# saglabāju tikai tās rindas, kur uzturēšanas klase ir norādīta (C, D, B)
uztur_klases = ['C', 'D', 'B']
df = df.loc[df['uzturesanas_klase'].isin(uztur_klases)]

In [27]:
# noņemu liekās atstarpes no nosaukumiem un aizpildu tukšās vērtības
df['nosaukums'] = df['nosaukums'].str.strip().replace('', pd.NA)

df.loc[:, 'nosaukums'] = df['nosaukums'].ffill()

In [28]:
# apskatu unikālās vērtības kolonnā 'segums'
df.segums.unique()

array(['grants', 'melnais', 'cits segums', 'bruģis', 'dubultā virsma',
       'Grants'], dtype=object)

In [29]:
# koriģēju dublikātus
df['segums'] = df['segums'].replace('Grants', 'grants')

In [30]:
# izveidoju jaunu kolonnu "atzars", lai identificētu atzaru posmus

df['atzars'] = df['nosaukums'].str.contains('atzars', case=False)
df['atzars'] = df['atzars'].fillna(False)

In [31]:
# pārbaudu, vai nav divi atzari pēc kārtas

df.loc[(df['atzars'].shift(-1) == True) & (df['nosaukums'].str.contains('atzars', case=False))].shape

(0, 7)

In [32]:
# pārnesu iepriekšējās rindas nosaukumu uz atzaru rindām

df.loc[df['atzars'] == True, 'nosaukums'] = df['nosaukums'].shift(1)

In [33]:
# piešķiru unikālu ID katram ceļam/ielai, kombinējot nosaukumu ar teritoriju
df['id'] = (df['nosaukums'] + df['teritorija']).factorize()[0] + 1

# pārbaudu kopējo ceļu/ielu skaitu
df['id'].nunique()

767

In [34]:
# pārbaudu datu tipus
df.dtypes

nosaukums            object
grupa                object
uzturesanas_klase    object
garums               object
segums               object
teritorija           object
atzars                 bool
id                    int64
dtype: object

In [35]:
# konvertēju kolonnu "garums" uz skaitlisku (float) formātu
df['garums'] = pd.to_numeric(df['garums'], errors="coerce")

In [36]:
# pārbaudu, vai visas kolonnas ir aizpildītas
df.isnull().sum()

nosaukums            0
grupa                0
uzturesanas_klase    0
garums               0
segums               0
teritorija           0
atzars               0
id                   0
dtype: int64

In [37]:
# sakārtoju indeksus un iegūstu sakārtotu datu kopu

df = df.reset_index(drop=True)
df

,nosaukums,grupa,uzturesanas_klase,garums,segums,teritorija,atzars,id
0,9410 A2- Kundziņi- Muldas,A ceļš,C,3.130,grants,Siguldas pagasts,False,1
1,9410 A2- Kundziņi- Muldas,A ceļš,C,0.880,grants,Siguldas pagasts,False,1
2,9420 Stūrīši- Ezermalas (Jūdaži),A ceļš,D,1.420,melnais,Siguldas pagasts,False,2
3,9433 P8- Limbas (Limbu ceļš),A ceļš,C,0.970,melnais,Siguldas pagasts,False,3
4,4205 Egļupes ceļš,A ceļš,C,0.420,grants,Allažu pagasts,False,4
...,...,...,...,...,...,...,...,...
1187,Upes iela,Iela,D,0.195,grants,Lēdurgas ciems,False,764
1188,Upeskalna iela,Iela,D,0.177,grants,Lēdurgas ciems,False,765
1189,Upeskalna iela,Iela,D,0.053,grants,Lēdurgas ciems,False,765
1190,Vītolu iela,Iela,D,0.200,grants,Lodes ciems,False,766
